## Fundamento matemático y estadístico


En este notebook se encuentran las definiciones de las funciones que sirven de soporte para el análisis matemático y estadístico. 

### Función de distribución acumulativa empírica 

O *Empirical cumulative distribution function* (ECDF) es una función de distribución asociada a la medida empírica de una muestra. En el eje X se ordenan los datos de menor a mayor, y en el eje Y se relaciona la fracción de datos que tienen un valor igual o menor que su correspondiente valor en X. 

In [3]:
import numpy as np

def ecdf(data):
    
    """Compute ECDF for a one-dimensional array of measurements."""
    # Number of data points: n
    n = len(data)

    # x-data for the ECDF: x
    x = np.sort(data)

    # y-data for the ECDF: y
    y = np.arange(1, n+1) / n

    return x, y

### Bootstrap

Técnica de remuestreo que consiste en:
1. Seleccionar aleatoriamente un elemento del dataset.
2. Copiar el valor de este elemento en otro conjunto de muestra. 
3. Repitir `n` veces los pasos anteriores, donde `n` es el número total de mediciones. 

En el último conjunto de muestra creado se realizan los cálculos estadísticos de interés, como puede ser la media, la moda, o el promedio. Los valores que se calculan sobre esta muestra son llamados *Bootstrap replicates*. 

In [13]:
def bootstrap_replicate_1d(data, func):
    """Generate bootstrap replicate of 1D data."""
    bs_sample = np.random.choice(data, len(data))
    return func(bs_sample)


def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates for 1d array"""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data, func)

    return bs_replicates

En regresiones lineales (cuyos datos vienen en pares), el método Bootstrap se aplica de la siguiente manera: 

1. Seleccionar aleatoriamente el par de datos X y Y.
2. Calcular la pendiente y el punto de corte para X y Y. 

Por lo tanto, cada pendiente y punto de corte calculado es un *Bootstrap replicate*. 

In [10]:
def draw_bs_pairs_linreg(x, y, size=1):
    """Perform pairs bootstrap for linear regression."""

    # Set up array of indices to sample from: inds
    inds = np.arange(0, len(x))

    # Initialize replicates: bs_slope_reps, bs_intercept_reps
    bs_slope_reps = np.empty(size)
    bs_intercept_reps = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_inds = np.random.choice(inds, size=len(inds))
        bs_x, bs_y = np.array(x)[bs_inds], np.array(y)[bs_inds]
        bs_slope_reps[i], bs_intercept_reps[i] = np.polyfit(bs_x, bs_y, deg=1)

    return bs_slope_reps, bs_intercept_reps


Una propiedad útil del método bootstrap es que los percentiles de los *bootstrap replicates* son equivalentes al intervalo de confianza, y pueden ser calculados con la función `np.percentile()`. A partir de estos intervalos, se definen los límites superiores e inferiores del rango de valores que pueden tomar los parámetros calculados si se repiten las mediciones muchas veces. 